In [62]:
import numpy as np
import qubovert as qv

In [100]:
A = np.array([[2,-1],[3,1]])
B = np.array([3,2])

In [122]:
def H(M, y, R = 2, a = -1, b = 3):
    '''
    M and y come from problem Mx-y
    R is precision, ie number of qubits
    a, b endpoints of desired range, a < b, [a,b)
    '''
    
    # Think I need Q to be upper triangular, how best to do this?
    # Need to add in the multiplication by pho part as well!!
    
    # Save the dimension of problem as N
    N = np.shape(y)[0]

    # Define the midpoint of the range, hard coded atm
    gamma = (b - a)/2
    pho = gamma ** 2
    
    #Intialise Q as vector of zeros 
    Q = np.zeros((N * R, N * R))
    
    # First term 
    for i in range(N):
        #print(f'i is {i}')
        for r in range(R):
            #print(f'r is {r}')
            for j in range(N):
                # print(f'j is {j}')
                for s in range(R):
                    # print(f's is {s}')
                    for k in range(N):
                        # print(f'k is {k}')
                        
                        # Update relevant entry of Q
                        Q[i * R + r,j * R + s] += pho * (2 ** (-r-s)) * M[k,i] * M[k,j]
                        
                        
    # Second term
    # Loop over each number 
    for i in range(N):
        # Loop over each qubit encoding said number
        for r in range(R):
            # Loop of k for matrix multiplication in index form
            for k in range(N):
                Q[i * R + r, i * R + r] += -2 * gamma * (2 ** (-r)) * M[k,i] * M[k,i]
                
    # Third term
    # Loop over each number
    for i in range(N):
        # Loop over j for the matrix multiplication
        for j in range(N):
            # Loop over each qubit encoding the number
            for r in range(R):
                Q[i * R + r, i * R + r] += -2 * gamma * (2 ** (-r)) * M[j,i] * y[j]
                
                
    # Make Q upper triangular
    
    
    return Q+np.triu(Q)-np.tril(Q)

H(A,B)    




Q = H(A,B, R = 8)
Q_qubo = qv.utils.matrix_to_qubo(Q)
#Q_qubo.solve_bruteforce()

qv.sim.anneal_qubo(Q_qubo, num_anneals = 10).best.state.values()



dict_values([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [126]:
def ConvertSolution(q, a, b):
    '''
    Function to convert an array of qubits into a real number
    '''
    
    # Store length of q vector
    n = np.shape(q)[0]
    
    # Initialise chi
    chi = 0
    
    for i in range(n):
        chi += (2 ** (-i)) * q[i]
                
    gamma = (b - a)/2
    
    x = gamma * chi + a
    
    return x
    
    
    
ConvertSolution([0, 1, 1, 1, 0, 0, 0, 0], -1, 3)

0.75